### Lectura de los parquet de contratos partición a partición

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

conf = (SparkConf().set("spark.executor.memory", "6g")
                    .set("spark.executor.cores", "3")
                    .set("spark.executor.instances", "2")
                    .set("spark.eventLog.enabled", "true"))
spark = SparkSession.builder.master("spark://spark-master:7077").appName("ContratosReadParquetPP").config(conf=conf).getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/07 17:31:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import time
print ('Inicio: '+time.strftime("%c"))
inicio = time.perf_counter()

Inicio: Mon Nov  7 17:31:36 2022


In [3]:
#spark job monitoring
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
from pyspark.sql.functions import sum, col, desc
import pyspark.sql.functions as F
from pyspark.sql import Window

esquema1 = StructType() \
      .add("ctId",StringType(),True) \
      .add("fecRef",StringType(),True) \
      .add("dato",StringType(),True) \
      .add("saldo",IntegerType(),True) \
      .add("proc",StringType(),True) \
      .add("fechIni",StringType(),True) \
      .add("fechFin",StringType(),True)


esquema2 = StructType() \
      .add("ctId",StringType(),True) \
      .add("fecRef",StringType(),True) \
      .add("dato",StringType(),True) \
      .add("saldo",IntegerType(),True) \
      .add("proc",StringType(),True)

for i in range(1, 16):
    inicioP = time.perf_counter()
    if i < 10: proc = 'CT000'+str(i)
    else:      proc = 'CT00'+str(i)

    print (132*'='+'\nTratando partición '+proc+' archivo con fechas')
    
    df = spark.read.option("mergeSchema", True).option("schema", "esquema1").option("basePath", "file:///opt/workspace/datos/contratosFecINIFIN.parquet/")\
               .format("parquet").load(['./datos/contratosFecINIFIN.parquet/proc='+proc])
    
    if i == 1:df.printSchema()
    print(df.count())
    df.show(10,truncate=False)
    print ('Fin lectura partición archivo con fechas: '+time.strftime("%c"))
    final = time.perf_counter()
    print(f'Hecho en {round(final - inicioP, 4)} segundo(s)')

    
    inicioP = time.perf_counter()

    print (132*'='+'\nTratando partición '+proc+' archivo original')

    df = spark.read.option("mergeSchema", True).option("schema", "esquema2").option("basePath", "file:///opt/workspace/datos/fakeContractsSinFechas.parquet/")\
               .format("parquet").load(['./datos/fakeContractsSinFechas.parquet/proc='+proc])    
    
    if i == 1:df.printSchema()
    print(df.count())
    df.show(10,truncate=False)
    print ('Fin lectura partición archivo original: '+time.strftime("%c"))
    final = time.perf_counter()
    print(f'Hecho en {round(final - inicioP, 4)} segundo(s)')

final = time.perf_counter()    
print(f'TOTAL hecho en {round(final - inicio, 4)} segundo(s)')



Tratando partición CT0001 archivo con fechas


root
 |-- ctId: string (nullable = true)
 |-- fecRef: string (nullable = true)
 |-- dato: string (nullable = true)
 |-- saldo: integer (nullable = true)
 |-- fechIni: string (nullable = true)
 |-- fechFin: string (nullable = true)
 |-- proc: string (nullable = true)



79285
+------------+----------+--------------------------------------------+-----+----------+----------+------+
|ctId        |fecRef    |dato                                        |saldo|fechIni   |fechFin   |proc  |
+------------+----------+--------------------------------------------+-----+----------+----------+------+
|CT0001123556|2020-03-19|Dato del contrato CT0001123556 en 2020-03-19|6800 |2020-03-19|2020-04-09|CT0001|
|CT0001123556|2020-04-09|Dato del contrato CT0001123556 en 2020-04-09|1200 |2020-04-09|2020-07-24|CT0001|
|CT0001123556|2020-07-24|Dato del contrato CT0001123556 en 2020-07-24|26800|2020-07-24|2020-12-19|CT0001|
|CT0001123556|2020-12-19|Dato del contrato CT0001123556 en 2020-12-19|9000 |2020-12-19|2021-02-02|CT0001|
|CT0001123556|2021-02-02|Dato del contrato CT0001123556 en 2021-02-02|45500|2021-02-02|2021-02-15|CT0001|
|CT0001123556|2021-02-15|Dato del contrato CT0001123556 en 2021-02-15|92900|2021-02-15|2021-02-23|CT0001|
|CT0001123556|2021-02-23|Dato del contra

In [4]:
spark.sparkContext.stop()
print('Sacabao')

Sacabao
